In [1]:
import os
import glob
import pandas as pd

In [2]:
csv_files = glob.glob("records/*/*trials.csv")

In [3]:
parzen_files = [f for f in csv_files if "parzen_sampler" in f]
grid_files   = [f for f in csv_files if "grid_sampler" in f]
random_files = [f for f in csv_files if "random_sampler" in f]

In [4]:
def make_dict(file_list):
    data_dict = {}
    for f in file_list:
        # Extract the middle portion: records/<key>/<file>
        key = os.path.basename(os.path.dirname(f))  # <--- folder name
        if "parzen_sampler" in key:
            data_dict[key.replace("_parzen_sampler", "")] = pd.read_csv(f)
        elif "grid_sampler" in key:
            data_dict[key.replace("_grid_sampler", "")] = pd.read_csv(f)
        elif "random_sampler" in key:
            data_dict[key.replace("_random_sampler", "")] = pd.read_csv(f)
    return data_dict

# Build dictionaries
parzen_dict = make_dict(parzen_files)
grid_dict   = make_dict(grid_files)
random_dict = make_dict(random_files)

In [5]:
from vizs import plot_efficiency_curves_best_of_N, plot_ard_panels

In [6]:
plot_efficiency_curves_best_of_N(parzen_dict, grid_dict, random_dict, exp_ks=(1,2,4,8,16,32))

In [7]:
plot_ard_panels(parzen_dict, grid_dict, random_dict, n_boot=64)

In [1]:
import argparse, os, math, json, random
from dataclasses import dataclass
from typing import Optional, List

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import optuna

# ---------------------------
# Utilities
# ---------------------------
def set_seed(seed:int):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

@dataclass
class PrepResult:
    X: np.ndarray
    preproc: str
    info: dict

def preprocess(trial, X: np.ndarray) -> PrepResult:
    choice = trial.suggest_categorical('preproc', ['raw','colnorm','pca'])
    info = {}
    Xp = X
    if choice == 'colnorm':
        thresh = trial.suggest_float('colnorm_thresh', 1e-9, 1e-3, log=True)
        std = X.std(axis=0, ddof=0)
        mask = std > thresh
        if mask.sum() == 0:
            mask = np.ones_like(std, dtype=bool)
        scaler = StandardScaler(with_mean=True, with_std=True)
        Xp = scaler.fit_transform(X[:, mask])
        info.update({'thresh': float(thresh), 'kept': int(mask.sum())})
    elif choice == 'pca':
        energy = trial.suggest_float('pca_energy', 0.5, 1.0)
        scaler = StandardScaler(with_mean=True, with_std=True)
        Xs = scaler.fit_transform(X)
        pca = PCA(n_components=energy, svd_solver='full')
        Xp = pca.fit_transform(Xs)
        info.update({'energy': float(energy), 'n_components': int(getattr(pca, "n_components_", Xp.shape[1]))})
    else:
        pass
    return PrepResult(X=Xp, preproc=choice, info=info)


# ---------------------------
# Model
# ---------------------------
class OneHiddenMLP(nn.Module):
    def __init__(self, n_in:int, n_hid:int, n_out:int, activation:str,
                 dist:str, scale_heur:str, scale_mult: Optional[float]):
        super().__init__()
        self.fc1 = nn.Linear(n_in, n_hid, bias=True)
        self.cl = nn.Linear(n_hid, n_hid, bias=False)
        self.fc2 = nn.Linear(n_hid, n_out, bias=True)

        if activation == 'tanh':
            self.act = nn.Tanh()
        elif activation == 'logistic':
            self.act = nn.Sigmoid()
        else:
            raise ValueError(activation)

        # zero-init final layer (zero_softmax analogue)
        nn.init.zeros_(self.fc2.weight); nn.init.zeros_(self.fc2.bias)

        # initialize first layer per requested heuristic/distribution
        if scale_heur == 'Glorot':
            nn.init.xavier_uniform_(self.fc1.weight) if dist == 'uniform' else nn.init.xavier_normal_(self.fc1.weight)
        elif scale_heur == 'old':
            fan_in = n_in
            s = (scale_mult if scale_mult is not None else 1.0) / math.sqrt(max(1.0, fan_in))
            if dist == 'uniform':
                nn.init.uniform_(self.fc1.weight, -s, s)
            else:
                nn.init.normal_(self.fc1.weight, 0.0, s)
        else:
            raise ValueError(scale_heur)
        nn.init.zeros_(self.fc1.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = self.act(self.fc1(x))
        c = self.cl(h)
        return self.fc2(h)

/opt/anaconda3/envs/sbn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
